# Mark-ICP tester

In [1]:
import os
import sys
import cv2
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/milestone_202110'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.geometry.geometry import GeometryItem
from pkg.geometry.geotype import GEOTYPE

from demo_utils.environment import add_cam, add_bed

from utils.detection_util import MultiICP, MODEL_DIR, ColorDepthMap

In [2]:
crob = CombinedRobot(robots_on_scene=[RobotConfig(0, RobotType.indy7, 
                           ((0,0,0), (0,0,0)),None)]
              , connection_list=[False])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
viewpoint = add_cam(gscene, tool_link="indy0_tcp")
VIEW_POSE = np.deg2rad([  0., 50.,  -70.,  -0.,  -90., 0])
gscene.show_pose(VIEW_POSE)
Tbc = viewpoint.get_tf(VIEW_POSE)

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### Get ICP result

In [3]:
color_image = cv2.imread(os.path.join(RNB_PLANNING_DIR, "data/test/color_0.jpg"), flags=cv2.IMREAD_UNCHANGED)
depth_mask = cv2.imread(os.path.join(RNB_PLANNING_DIR, "data/test/depth_mask_0.png"), flags=cv2.IMREAD_UNCHANGED)

cdp = ColorDepthMap(color_image, depth_mask, 
                    [1280, 720,
                     909.957763671875, 909.90283203125,
                     638.3824462890625, 380.0085144042969], 
                    1 / 3999.999810010204)
cameraMatrix = np.array([[cdp.intrins[2], 0, cdp.intrins[4]],
                         [0, cdp.intrins[3], cdp.intrins[5]],
                         [0, 0, 1]])
distCoeffs = np.array([0]*5)

icp_bed = MultiICP(model=MODEL_DIR + '/bed/bed.STL', 
                   Toff=SE3([[0,1,0],[0,0,1],[1,0,0]], [0.455,0,1.02]))

icp_bed.clear()
pcd = icp_bed.add_image(cdp, Tc=None)


### Show on RVIZ

In [4]:
gscene.create_safe(GEOTYPE.MESH, "bed_pcd", "base_link", (0.1,0.1,0.1), (0,0,0), rpy=(0,0,0), color=(1,0,0,1), display=True,
                   collision=False, fixed=True, scale=(0.01,0.01,1),
                   vertices=np.matmul(np.asarray(pcd.points), Tbc[:3,:3].transpose())+Tbc[:3,3])

[WARN] Vertices for mesh should be have center point (0,0,0). Auto adjusting.


### Compute ICP

In [5]:
ICP_result, fitness = icp_bed.compute_ICP(To=SE3(Rot_axis_series([3, 2],[-np.pi/2, np.pi/2]), (0,0,3)), visualize=True)
Tdetect=np.matmul(Tbc, ICP_result)

Apply point-to-point ICP
registration::RegistrationResult with fitness=4.853144e-01, inlier_rmse=4.227020e-02, and correspondence_set size of 16507
Access transformation to get result.
Transformation is:
[[ 0.90990316 -0.06958906  0.40894205 -0.42510747]
 [-0.19457271 -0.9422517   0.2725861   0.02932281]
 [ 0.36635733 -0.32759592 -0.87089794  4.3809776 ]
 [ 0.          0.          0.          1.        ]]


### Visualize and adjust
* Adjust Tdetect to get perfect GT

In [6]:
bed = add_bed(gscene, Tdetect[:3,3], Rot2rpy(Tdetect[:3,:3]), (0,1,0,0.5))

# Make ArucoMap for Testing

### Testing single marker
* Make single-marker aruco map to get one-marker transformation

In [7]:
from cv2 import aruco
from pkg.detector.aruco.detector import *

def get_aruco_map_test(test_name, mk_idx, mk_size, point=[0,0,0], direction=(0,0,0)):
    dictionary = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)
    #     params = aruco.DetectorParameters_create()

    aruco_map = ArucoMap(dictionary=dictionary, _dict={
        test_name: MarkerSet(test_name,
                         dlevel=DetectionLevel.ENVIRONMENT, gtype=GEOTYPE.BOX, dims=(0.1, 0.1,0.1), color=(0.8,0.0,0.0,1),
                         _list=[
                             ObjectMarker(test_name, mk_idx, mk_size, point, direction)
                         ])
    })
    return aruco_map

In [8]:
mk_idx = 230
mk_size = 0.15
aruco_map = get_aruco_map_test("bed_test", mk_idx, mk_size)
objectPose_dict, corner_dict = aruco_map.get_object_pose_dict(color_image, cameraMatrix, distCoeffs)
Tbmark = objectPose_dict['bed_test']
Tbmark = np.matmul(Tbc, Tbmark)

### Show marker location on RVIZ

In [9]:
gscene.add_highlight_axis("hl", "mark_{}".format(mk_idx), T=Tbmark)

### Get relative marker offset

In [10]:
Tmark = np.matmul(SE3_inv(Tdetect), Tbmark)

point = Tmark[:3,3]
direction = Rot2rpy(Tmark[:3,:3])
print("point: {}, {}, {}".format(*point))
print("direction: {}, {}, {}".format(*direction))

point: 1.20029355972, -0.331169991039, 0.173714289576
direction: -1.65989209067, 0.0468366767868, 1.59556876223


### Visualize result

In [11]:
aruco_map = get_aruco_map_test("bed_test", mk_idx, mk_size, point=point, direction=direction)
objectPose_dict, corner_dict = aruco_map.get_object_pose_dict(color_image, cameraMatrix, distCoeffs)
Tbed = objectPose_dict['bed_test']
Tbed = np.matmul(Tbc, Tbed)

In [12]:
add_bed(gscene, Tbed[:3,3], Rot2rpy(Tbed[:3,:3]), (0,1,0,0.5))